In [1]:
"""
This code takes the situations from the situations file and sorts them accordig to difficulty and difference
in log-likelihoods.
"""
import pandas as pd
import os
import numpy as np
from ast import literal_eval

In [2]:
df = pd.DataFrame()
for i in range(3000):
    filepath = f"situations/situations_with_llhs_{i}_v6.csv"
    if os.path.isfile(filepath):
        df = df.append(pd.read_csv(filepath))

llh_types = ['llh_sparsemax_to_sparsemax', 'llh_sparsemax_to_lqr', 'llh_sparsemax_to_nm2', 'llh_lqr_to_sparsemax', 'llh_lqr_to_lqr', 'llh_lqr_to_nm2', 'llh_nm2_to_sparsemax',
            'llh_nm2_to_lqr', 'llh_nm2_to_nm2']

def get_min_llh_diff(row):
    llh_diffs = []
    for llh_type_1 in llh_types:
        for llh_type_2 in llh_types[llh_types.index(llh_type_1):]:
            llh_diffs.append(abs(row[llh_type_1] - row[llh_type_2]))
        
    return min(llh_diffs)

def get_sum_llh_diff(row):
    llh_diffs = []
    for llh_type_1 in llh_types:
        for llh_type_2 in llh_types[llh_types.index(llh_type_1):]:
            llh_diffs.append(abs(row[llh_type_1] - row[llh_type_2]))
        
    return sum(llh_diffs)

df['min_llh_diff'] = df.apply(get_min_llh_diff, axis=1)

df['sum_llh_diff'] = df.apply(get_sum_llh_diff, axis=1)

n_params_sparsemax = 4
n_params_nm2 = 2
n_params_lqr = 2

df['AIC_sparsemax_to_sparsemax'] = 2 * n_params_sparsemax - 2 * df['llh_sparsemax_to_sparsemax']
df['AIC_sparsemax_to_lqr'] = 2 * n_params_lqr - 2 * df['llh_sparsemax_to_lqr']
df['AIC_sparsemax_to_nm2'] = 2 * n_params_nm2 - 2 * df['llh_sparsemax_to_nm2']

df['AIC_lqr_to_sparsemax'] = 2 * n_params_sparsemax - 2 * df['llh_lqr_to_sparsemax']
df['AIC_lqr_to_lqr'] = 2 * n_params_lqr - 2 * df['llh_lqr_to_lqr']
df['AIC_lqr_to_nm2'] = 2 * n_params_nm2 - 2 * df['llh_lqr_to_nm2']

df['AIC_nm2_to_sparsemax'] = 2 * n_params_sparsemax - 2 * df['llh_nm2_to_sparsemax']
df['AIC_nm2_to_lqr'] = 2 * n_params_lqr - 2 * df['llh_nm2_to_lqr']
df['AIC_nm2_to_nm2'] = 2 * n_params_nm2 - 2 * df['llh_nm2_to_nm2']

df['sparsemax_generated_posterior'] = df.apply(lambda x: np.exp(-x['AIC_sparsemax_to_sparsemax']/2) / (np.exp(-x['AIC_sparsemax_to_sparsemax']/2) + np.exp(-x['AIC_sparsemax_to_lqr']/2 + np.exp(-x['AIC_sparsemax_to_nm2']))), axis=1)
df['lqr_generated_posterior'] = df.apply(lambda x: np.exp(-x['AIC_lqr_to_lqr']/2) / (np.exp(-x['AIC_lqr_to_sparsemax']/2) + np.exp(-x['AIC_lqr_to_lqr']/2) + np.exp(-x['AIC_lqr_to_nm2']/2)), axis=1)
df['nm2_generated_posterior'] = df.apply(lambda x: np.exp(-x['AIC_nm2_to_nm2']/2) / (np.exp(-x['AIC_nm2_to_sparsemax']/2) + np.exp(-x['AIC_nm2_to_lqr']/2) + np.exp(-x['AIC_nm2_to_nm2']/2)), axis=1)
df['posterior_sum'] = df.apply(lambda x: x['sparsemax_generated_posterior'] + x['lqr_generated_posterior'] + x['nm2_generated_posterior'], axis=1)

df.sort_values(by='sum_llh_diff', ascending=False).to_csv("situations_by_informativeness_sum.csv")
df.sort_values(by='sum_llh_diff', ascending=False).to_csv("situations_by_informativeness_min.csv")
df.sort_values(by='sparsemax_score_noisy').to_csv('situations_by_difficulty.csv')

In [3]:
df.sort_values(by='posterior_sum', ascending=False)

,Unnamed: 0,situation,llh_sparsemax_to_lqr,llh_sparsemax_to_sparsemax,llh_sparsemax_to_nm2,llh_lqr_to_sparsemax,llh_lqr_to_lqr,llh_lqr_to_nm2,llh_nm2_to_sparsemax,llh_nm2_to_lqr,...,AIC_lqr_to_sparsemax,AIC_lqr_to_lqr,AIC_lqr_to_nm2,AIC_nm2_to_sparsemax,AIC_nm2_to_lqr,AIC_nm2_to_nm2,sparsemax_generated_posterior,lqr_generated_posterior,nm2_generated_posterior,posterior_sum
0,0,"[241.0, -127.0, -192.0, 113.0, -224.0]",-131.379380,-95.779808,-126.083878,-68.134008,-48.077925,-68.133654,-60.999082,-111.642632,...,144.268016,100.155849,140.267308,129.998165,227.285264,120.323029,1.0,1.000000e+00,0.992136,2.992136
0,0,"[95.0, -108.0, -140.0, 62.0, -173.0]",-111.076572,-78.097419,-103.027215,-107.423048,-47.570660,-106.517969,-60.343969,-110.229395,...,222.846096,99.141320,217.035938,128.687938,224.458790,120.622162,1.0,1.000000e+00,0.982586,2.982586
0,0,"[-84.0, 219.0, 125.0, -84.0, 132.0]",-196.518570,-112.867122,-184.121025,-61.107707,-51.531093,-59.290741,-58.707539,-113.286208,...,130.215414,107.062186,122.581482,125.415077,230.572417,117.352783,1.0,9.995642e-01,0.982556,2.982120
0,0,"[-88.0, 3.0, 58.0, 82.0, 231.0]",-117.268793,-90.053350,-113.035334,-64.809907,-54.837799,-80.749095,-62.902245,-114.488378,...,137.619813,113.675598,165.498191,133.804490,232.976757,126.199174,1.0,9.999937e-01,0.978176,2.978169
0,0,"[-78.0, -67.0, -202.0, -177.0, -199.0]",-116.777503,-98.660216,-118.466515,-103.609268,-96.334536,-104.453950,-62.891793,-114.631639,...,215.218535,196.669071,212.907899,133.783586,233.263278,126.564288,1.0,9.996087e-01,0.973652,2.973260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0,"[179.0, 44.0, -156.0, 220.0, 117.0]",-155.369025,-108.969555,-151.581547,-70.016241,-74.689462,-67.639110,-53.320040,-112.552368,...,148.032482,153.378925,139.278221,114.640079,229.104737,110.633794,1.0,8.556141e-04,0.881127,1.881982
0,0,"[229.0, -174.0, 51.0, 62.0, 164.0]",-123.630940,-95.429437,-120.551058,-54.974426,-69.495292,-50.470715,-54.605878,-116.426211,...,117.948852,142.990583,104.941431,117.211756,236.852422,113.189733,1.0,5.458604e-09,0.881948,1.881948
0,0,"[205.0, -239.0, -128.0, 182.0, 233.0]",-169.353990,-113.792948,-174.412477,-55.248946,-67.781278,-49.481102,-55.549462,-119.120823,...,118.497891,139.562555,102.962205,119.098923,242.241646,115.191921,1.0,1.127590e-08,0.875828,1.875828
0,0,"[-115.0, -199.0, 226.0, -29.0, 80.0]",-136.484613,-100.690251,-131.472550,-69.391281,-107.113172,-68.892318,-54.728817,-112.670626,...,146.782562,218.226344,141.784636,117.457634,229.341251,113.618425,1.0,2.325939e-17,0.872094,1.872094


In [4]:
len(df[df['lqr_generated_posterior'] > 0.5])

1048

In [5]:
df['lqr_generated_posterior'].mean()

0.35216173753197333

In [6]:
df['sparsemax_generated_posterior'].mean()

0.999999988221553

In [7]:
df['ease'] = df['sparsemax_score_noisy'] - df['lqr_score']

In [8]:
df_selected = df.sort_values(by='posterior_sum', ascending=False)[:20].append(df.sort_values(by='ease', ascending=True)[:10])

In [9]:
df_selected['condition'] = ['informative' for _ in range(20)] + ['easy' for _ in range(10)]

In [10]:
df_selected['situation'] = df_selected['situation'].apply(lambda x: [int(np.round(e)) for e in literal_eval(x)])

In [15]:
df_selected['root_cost'] = df_selected['sparsemax_score_noisy'].apply(np.sqrt)

In [16]:
df_selected[df_selected['condition'] == 'easy']['root_cost'].median()

131.55465428371008

In [17]:
df_selected[df_selected['condition'] == 'informative']['root_cost'].median()

1309.9458962700937

In [18]:
df_selected

,Unnamed: 0,situation,llh_sparsemax_to_lqr,llh_sparsemax_to_sparsemax,llh_sparsemax_to_nm2,llh_lqr_to_sparsemax,llh_lqr_to_lqr,llh_lqr_to_nm2,llh_nm2_to_sparsemax,llh_nm2_to_lqr,...,AIC_nm2_to_sparsemax,AIC_nm2_to_lqr,AIC_nm2_to_nm2,sparsemax_generated_posterior,lqr_generated_posterior,nm2_generated_posterior,posterior_sum,ease,condition,root_cost
0,0,"[241, -127, -192, 113, -224]",-131.379380,-95.779808,-126.083878,-68.134008,-48.077925,-68.133654,-60.999082,-111.642632,...,129.998165,227.285264,120.323029,1.0,1.000000e+00,0.992136,2.992136,1.259040e+06,informative,1122.130964
0,0,"[95, -108, -140, 62, -173]",-111.076572,-78.097419,-103.027215,-107.423048,-47.570660,-106.517969,-60.343969,-110.229395,...,128.687938,224.458790,120.622162,1.0,1.000000e+00,0.982586,2.982586,1.206853e+06,informative,1098.578809
0,0,"[-84, 219, 125, -84, 132]",-196.518570,-112.867122,-184.121025,-61.107707,-51.531093,-59.290741,-58.707539,-113.286208,...,125.415077,230.572417,117.352783,1.0,9.995642e-01,0.982556,2.982120,4.264606e+05,informative,653.046660
0,0,"[-88, 3, 58, 82, 231]",-117.268793,-90.053350,-113.035334,-64.809907,-54.837799,-80.749095,-62.902245,-114.488378,...,133.804490,232.976757,126.199174,1.0,9.999937e-01,0.978176,2.978169,6.030499e+05,informative,776.576912
0,0,"[-78, -67, -202, -177, -199]",-116.777503,-98.660216,-118.466515,-103.609268,-96.334536,-104.453950,-62.891793,-114.631639,...,133.783586,233.263278,126.564288,1.0,9.996087e-01,0.973652,2.973260,2.243272e+06,informative,1497.760829
0,0,"[-70, 30, 186, -238, 218]",-182.895252,-116.515532,-172.311796,-64.299682,-58.673284,-62.920299,-60.274351,-114.588274,...,128.548703,233.176548,119.988474,1.0,9.854214e-01,0.986348,2.971769,3.277901e+06,informative,1810.506697
0,0,"[-203, -241, 137, -218, -143]",-184.008581,-118.093209,-172.906541,-109.005480,-75.909573,-110.998864,-54.935234,-116.490611,...,117.870469,236.981221,110.898830,1.0,1.000000e+00,0.970282,2.970282,6.678005e+05,informative,817.334326
0,0,"[-140, 17, 95, 220, 85]",-128.149939,-88.987051,-116.073944,-76.740899,-43.909063,-75.674413,-59.404025,-111.930908,...,126.808051,227.861815,119.961127,1.0,1.000000e+00,0.968430,2.968430,3.842349e+06,informative,1960.205806
0,0,"[204, -158, 121, -59, 25]",-115.716264,-88.778103,-108.433108,-85.456029,-59.641065,-98.183343,-55.377190,-110.673325,...,118.754380,225.346649,112.002707,1.0,1.000000e+00,0.966941,2.966941,2.566060e+06,informative,1601.902681
0,0,"[111, -133, -75, 92, -149]",-109.506717,-80.631800,-98.621001,-58.532158,-44.969192,-57.706045,-56.755144,-109.386975,...,121.510288,222.773950,114.759897,1.0,9.999969e-01,0.966920,2.966917,4.857608e+05,informative,696.979598


In [14]:
df_selected.to_csv('selected_situations_e3_final_full.csv', index=False)

In [20]:
df_selected[['situation', 'condition']].to_csv('selected_situations_e3_final.csv', index=False)